<body>
    <div>
        <h2 style='font-family: cursive; font-size: 40px;'>Método del Gradiente conjugado</h2>
<!--         <img src= alt=middle style="height: 340px"> -->
    </div>
</body>

<h3 style="font-family: 'Courier New'; font-size:25px"
    >Formulación matemática</h3>

<div>
$x^{k+1} = x^{k} + \alpha_k * d^k$
<br>
    
$
d^{k+1} =
\begin{cases}
    -g \quad\quad\quad\quad\quad\quad si\quad k+1=0 \\
    -g^{k+1} + \beta_k * d^k \quad si\quad k \gt 0
\end{cases}
$
</div>

<h3 style="font-family: 'Courier New'; font-size:25px"
    >Valores iniciales</h3>

Vector inicial: Vector con los valores iniciales para la primer iteración.
<br>
Tol: Exactitud que debe tener la aproximación.
<br>
Regla: Regla con la que se va a calcular el $\beta_k$.



<h3 style="font-family: 'Courier New'; font-size:25px"
    >Ventajas y desventajas</h3>

<table style="width:100%; text-align: left; font-size: 14px">
    <tr style="text-align: left;">
        <th style="text-align: left;">Ventajas</th>
        <th style="text-align: left;">Desventajas</th>
    </tr>
    <tr>
        <td style="text-align: left;">No requiere algebra lineal numerica.</td>       
        <td style="text-align: left;">Tiene el inconveniente de converger lentamente.</td>
    </tr>
    <tr>
        <td style="text-align: left;">Utilizapoca memoria a problemas a gran escala.</td>
        <td style="text-align: left;">Los pasos suelen tener una reducida escala de longitud.</td>
    </tr>
</table>

<h3 style="font-family: 'Courier New'; font-size:25px"
    >Pseudocódigo</h3>

<ol>
    <li>g=$\nabla f(x^{(0)})^t$</li>
    <li>$d^{(0)}=-g^{(0)}$</li>
    <li>Para error > tol</li>
    <li>&emsp;$\alpha_k=1$; $\delta \epsilon ]0,1[$</li>
    <li>&emsp;Mientras $f(x_k+\alpha_k*d_k)-f(x_k)\gt \delta * \alpha_k * (g_k)^t * d_k$</li>
    <li>&emsp;&emsp;$\alpha_k=\alpha_k/2$</li>
    <li>&emsp;Fin del Mientras</li>
    <li>&emsp;$x^{k+1} = x^k + \alpha_k*d^k$</li>
    <li>&emsp;error = $\|$ $\nabla f(x_1^{k},$ $...,$ $x_{j-1}^{k},$ $x_{j},$ $x_{j+1}^{k-1},$ $x_{n}^{k-1})$ $\|$</li>
    <li>&emsp;Si error $\lt$ tol</li>
    <li>&emsp;&emsp;Retornar x^{k+1}</li>
    <li>&emsp;Fin Si</li>
    <li>&emsp;$g^{k+1} = \nabla f(x^{k+1})^t$</li>
    <li>&emsp;Calcula $\beta_k$ con la regla seleccionada</li>
    <li>&emsp;d^{k+1} = -g^{k+1} + \beta_k * d^k</li>
    <li>Fin del Para</li>
</ol>

<h3 style="font-family: 'Courier New'; font-size:25px">Octave</h3>

In [7]:
format long


function [g] = evaluar_gradiente(gradiente, varsym, vector)
    %    Funcion para evaluar el gradiente con un vector ingresado
    %    :param gradiente: gradiente que se debe evaluar
    %    :param varsym: lista con las variables simbolicas de la ecuacion
    %    :param vector: vector que se debe evaluar en el gradiente
    %    :return: resultado de evaluar el vector en el gradiente
    g = [];
    for i = 1:length(varsym)
        g = [g double(subs(gradiente(i), varsym, vector))];
    end
end

function [gradiente]=calcular_gradiente(footion,vars)
    %Esta funcion calcula el gradiente de la funcion de entrada.
    %
    %Sintaxis: calcular_gradiente(f,vars)
    %
    %Parámetros Iniciales:
    %            f = una cadena de caracteres (string) que representa a la función f.
    %            vars = Variables de la funcion.
    %
    %Parámetros de Salida: 
    %            gradiente = El gradiente de la funcion.
    %
    n=length(vars);    
    for i=1:n
        variables(i)=sym(vars(i));
    end
    for i=1:n
        % Deriva la funcion respecto a cada variable de la lista variables
        gradiente(i)=diff(footion, variables(i));
    end
end


function [B_k] = seleccionaB(g, d, symvars, vectork, vector_k1, regla)
    %    Funcion para calcular la regla_bk
    %    :param g: vector gk
    %    :param d: vector dk
    %    :param vectork: vector k
    %    :param vector_k1: vector k+1
    %    :param regla: regla deseada para calcular bk
    %    :return: valor de bk calculado
    if regla == 'FR'
        % Metodo de Fletcher and Reeves
        B_k = norm(double(subs(g, symvars, vector_k1)))**2/norm(double(subs(g, symvars, vectork)))**2;
    elseif regla == 'CD'
        % Metodo de Descenso Conjugado CD
        B_k = norm(double(subs(g, symvars, vector_k1)))**2/(sum((-d) .* double(subs(g, symvars, vectork))));
    elseif regla == 'DY'
        % Metodo de Dai and Yuan
        yk = vector_k1 - vectork;
        B_k = norm(double(subs(g, symvars, vector_k1)))**2/(sum(d .* yk));
    end
end

function [vector_k1 e] = gradiente_conjugado(f, variables, vectork, tol, iter, regla)
    % Funcion que implementa el metodo de gradiente conjugado
    % :param f: string con la funcion que se debe evaluar
    % :param variables: lista con las variables de la ecuacion
    % :param vectork: vector inicial de necesita el metodo
    % :param tol: tolerancia al fallo que debe tener el resultado
    % :param iter: iteraciones maximas que no debe superar el metodo
    % :param regla que va dar valor a B_k
    % :return: lista con dos elementos, vector calculado y lista de error
    warning('off', 'all');
    pkg load symbolic
    
    footion = str2func(f);
    n = length(variables);
    symvars = [];
    e = [];
    
    for i=1:n
        symvars = [symvars sym(variables(i))];
    end
    g = calcular_gradiente(footion, symvars);
    %Obtiene los valores iniciales de g y d
    g_num = evaluar_gradiente(g, symvars, vectork);
    d = -g_num;
    
    for k=1:iter
        alpha_k = 1;
        %Evalua la condicion hasta que encuentre un alpha_k que pueda cumplir la condicion
        while double(subs(footion, symvars, vectork + alpha_k * d)) - double(subs(footion, symvars, vectork)) >= double(0.5*alpha_k*sum(g_num .* d))
            %Calcula el nuevo alpha
            alpha_k = double(alpha_k / 2);
        end
        %Calcula el nuevo valor del vector k
        vector_k1 = vectork + alpha_k * d;
        error = double(norm(vector_k1 - vectork));
        e = [e error];
        if error < tol
            break;
        end
        %Calcula el nuevo g
        g_num = evaluar_gradiente(g, symvars, vector_k1);
        %Calcula el nuevo B_k
        B_k = double(seleccionaB(g, d, symvars, vectork, vector_k1, regla));
        %Calcula el nuevo valor de d
        d = double(-g_num + B_k*d);
        vectork = vector_k1;
    end
end 

<h3 style="font-family: 'Courier New'; font-size:25px">Python</h3>

In [8]:
from sympy import symbols, sympify, diff, lambdify, Matrix
from sympy.matrices import matrix_multiply_elementwise

def calcular_funciones(f,vars):
    """
    Esta funcion calcula el gradiente de la funcion de entrada.
    Parametros iniciales:
    f = una cadena de caracteres (string) que representa a la función f.
    vars = variables de la funcion.
    
    Parámetros de Salida: 
    deetions = el gradiente de la funcion.
    """
    svars = symbols(vars)
    footion = sympify(f)
    deetions = []
    for var in svars:
        #Deriva la funcion respecto a cada variable
        deetions.append(diff(footion, var))
    return footion, deetions, svars


def evaluar_gradiente(gradiente, varsym, vector):
    """
    Funcion para evaluar el gradradiente con un vector ingresado
    :param gradiente: gradiente que se debe evaluar
    :param varsym: lista con las variables simbolicas de la ecuacion
    :param vector: vector que se debe evaluar en el gradiente
    :return: resultado de evaluar el vector en el gradiente
    """
    g = []
    #Obtiene un diccionario de las variables:valores
    vals = dict(zip(varsym, vector))
    for i in range(len(varsym)):
        g.append(gradiente[i].subs(vals))
    return Matrix(g)
    
    
def seleccionaB(g, gk, d, svars, vectork, vector_k1, regla):
    """
    Funcion para calcular la regla_bk
    :param g: vector gk de la iteracion anterior
    :param gk: vector gk
    :param d: vector d
    :param svars: variables de la funcion
    :param vectork: vector k
    :param vector_k1: vector k+1
    :param regla: regla deseada para calcular bk
    :return: valor de B_k calculado
    """
    if regla == 'FR':
        # Metodo de Fletcher and Reeves
        B_k = gk.norm(2)**2/g.norm(2)**2
    elif regla == 'CD':
        # Metodo de Descenso Conjugado CD
        B_k = gk.norm(2)**2/sum(-matrix_multiply_elementwise(d, g))
    elif regla == 'DY':
        # Metodo de Dai y Yuan
        yk = vector_k1 - vectork
        B_k = gk.norm(2)**2/(sum(matrix_multiply_elementwise(d, yk)))
    else:
        print('La regla ', regla, ' no es una regla valida')
    return B_k
    
    
def gradiente_conjugado(f, variables, vectork, tol, iterMax, regla):
    footion, gradiente, svars = calcular_funciones(f, variables)
    e = []
    g = evaluar_gradiente(gradiente, svars, vectork)
    d = -g
    error = tol + 1
    vectork = Matrix(vectork)
    
    for i in range(iterMax):
        alpha_k = 1
        while footion.subs(dict(zip(svars, vectork + alpha_k * d))) - footion.subs(dict(zip(svars, vectork))) >= 0.5 * alpha_k * sum(matrix_multiply_elementwise(g, d)):
            alpha_k /= 2
        
        vectork1 = vectork + alpha_k * d
        error = (vectork1 - vectork).norm(2)
        e.append(error)
        if error < tol:
            break
        gk = evaluar_gradiente(gradiente, svars, vectork1)
        B_k = seleccionaB(g, gk, d, svars, vectork, vectork1, regla)
        d = -gk + B_k * d
        vectork = vectork1
        g = gk
                                
    return vectork

<h3 style="font-family: 'Courier New'; font-size:25px">Ejemplo Numérico</h3>

<h5 style="font-family: 'Courier New'; font-size:16px">Python</h5>

In [9]:
print(gradiente_conjugado('(x - 2) ** 2 + (y + 3) ** 2 + x * y', ['x', 'y'], [1, 1], 10^(-10), 10, 'CD'))    
    
print(gradiente_conjugado('x ** 3 + y ** 3 + z ** 3 - 2 * x * y - 2 * x * z - 2 * y * z', ['x', 'y', 'z'], [1, 1, 1], 10^(-10), 10, 'DY'))

Matrix([[4.66647106708098], [-5.33313444327900]])


NameError: name 'vector_k1' is not defined

<h5 style="font-family: 'Courier New'; font-size:16px">Octave</h5>

In [6]:
[res, e] = gradiente_conjugado('@(x,y,z)x ** 3 + y ** 3 + z ** 3 - 2 * x * y - 2 * x * z - 2 * y * z', ['x', 'y', 'z'], [1, 1, 1], 10^(-10), 10, 'FR')
    

Symbolic pkg v2.8.0: Python communication link active, SymPy v1.5.1.
res =

   1.333331776323713   1.333331776323713   1.333331776323713

e =

 Columns 1 through 3:

   4.330127018922193e-01   8.880143300524028e-02   4.134592346876464e-02

 Columns 4 through 6:

   9.851792774233575e-03   3.093794738757982e-03   8.769852209933776e-04

 Columns 7 through 9:

   2.604984529655185e-04   7.567994980346868e-05   2.225586349537787e-05

 Column 10:

   6.507003381375837e-06

